In [79]:
import pandas as pd
import numpy as np
import folium
import simplejson,requests
import sys
import time
import import_ipynb
import YouTubeAPI as youtube
import pymysql as db

In [80]:
import konlpy
from konlpy.corpus import kolaw
from konlpy.tag import *
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import copy

In [81]:
local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
con = db.connect(
    host=local,
    user='admin',
    db='Nexpot',
    password='qwertyuiop123',
    charset='utf8'
)
cur = con.cursor()

In [98]:
def cleanText(readData): 
# url 제거
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',readData)
# 이메일 제거    
    text = re.sub('[-A-Za-z0-9_]+[-A-Za-z0-9_.][@]{1}[-A-Za-z0-9_]+[-A-Za-z0-9_.][.]{1}[A-Za-z]{1,5}',' ',text)
#텍스트에 포함되어 있는 특수 문자 제거    
    text = re.sub('[:=+,#/\?^$.@*\"※~&%ㆍ!』\\‘’|\(\)\[\]\<\>`\'…》▶✨✔✅❤★♡️♥ㅂㅈㄷㄱㅅㅁㄴㅇㄹㅎㅋㅌ\
                  ㅊㅍㅛㅕㅑㅐㅔㅗㅓㅏㅣㅠㅜㅡ]', ' ', text) 
    text = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE).sub(r' ',text)   # 이모티콘 제거
    text = re.sub('\\n\\t\\r★',' ',text)                         # 탭, 줄띄우기 등 제거
    text = re.sub(youtube.channelname ,'', text)
    return text
print(cleanText(desc[1]))
# 특정 이모티콘 제거 불가 :🌊 🏖 ✉ 등

 뚜벅이여행  여행  여행코스  가볼만한곳  국내여행  핫플  유월드루지테마파크  웅천해수욕장  장도  예술랜드  하이클래스  해상케이블카  미남크루즈

  Instagram - uiland kr
✉  Email -  

영상에 뚜벅이를 위한 전라남도 
필수여행코스 Best 7   핫플레이스

그리고 여행경비와 교통정보까지
모두 나와있으니 참고해주세요 

 전라남도  하루 여행코스 

 유월드루지테마파크
  -  전남 시 소라면 안심산길 155

 웅천해수욕장
  -  전남 시 예울마루로 37-40

 장도
  -  전남 시 웅천동 산 274

 예술랜드
  -  전남 시 돌산읍 무술목길 142-1

 카페하이클래스
  -  전남 시 돌산읍 무술목길 114

 해상케이블카
  -  전남 시 돌산읍 돌산로 3600-1  돌산정류장 
  -  전남 시 자산4길 39                   자산정류장 

 미남크루즈
  -  전남 시 돌산읍 돌산로 3617-22

----------------------------------------------------------------------------------

  노래   효과음

Epidemic Sound -  

가입 시 한 달 무료로 사용이 가능합니다  

----------------------------------------------------------------------------------

  촬영장비

Gopro Hero Black 9
Gopro Hero Black 10
iPhone 12 Pro Max

  편집장비

2021 맥북프로 16인치
2021 맥북프로 14인치


In [102]:
n = 7
desc = youtube.description
print(desc[n])
ex_desc = desc[n].split('\n')  # 각 줄마다 split
lex_desc = []
for i in ex_desc:
    # 지번주소 제거
    if bool(re.search('(([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d-]+)|(([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d][^시]+)',i)):
        continue
    # 도로명주소 제거
    if bool(re.search('(([가-힣A-Za-z·\d~\-\.]{2,}(로|길).[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동|면)\s)[\d]+)',i)):
        continue
    else:
        lex_desc.append(i)
spl_place = []
for a in lex_desc:                      # 타임라인에 존재하는 장소 추출
    if bool(re.search('^\d{1,2}:\d{2}',a)):
        sub_place = re.sub('^\d{1,2}:\d{2} ?','', a)
        spl_place.append(sub_place.lower())
    else:
        continue
print(spl_place)
place = []
for j in spl_place:
    i = j
    if '인트로' in i or 'intro' in i:                    # Intro는 제거
        continue
    if '먹방' in i or '여행' in i or '맛집' in i:
        continue
    if "(" in i:
        i = i.split("(")[0]
    if "#" in i:
        i = i.split("#")[1]  # 뒤에 존재하는 장소 추출
    if "/"in i:
        a = i.split("/")[0]  # / 뒤에 존재하는 장소 추출
        b = i.split("/")[1]
        place.append(a)
        place.append(b)
        continue
    place.append(i)
print(place)
remove_address_place = " ".join(lex_desc)
if len(place) < 2:
    hannanum = Hannanum()              # Mecab 쓰고 싶은데... 설치를 다시 해도 오류가 뜸..;;
    h_place = hannanum.nouns(cleanText(remove_address_place))
    num_place = copy.deepcopy(h_place)
    site = []
    for i in h_place:
        if str.isdigit(i):
            num_place.remove(i)
            continue
        if '여행' in i:
            num_place.remove(i)
            continue
        if len(i) == 1:
            num_place.remove(i)
    place = num_place
    print(h_place)
    print(num_place)

🐻영상 속 정보
- 마스크 : 참존 톤업핏 베이지
- 옷,악세사리 : 레이지코너
- 숙소 : 여수 라테라스 리조트
- 쿠션 : 라네즈 네오 쿠션

🐻맛집 & 여행 정보
- 맛집 : 
첫째날 : 황소식당 (돌게장 맛집), 41번가 포차(숙성회 맛집)
둘째날 : 조롱박(아구찜, 서대회 맛집), 호남갈비 (갈비맛집), 여수 낭만 포차 거리
셋째날 : 꽃돌게장 1번가 (게장 맛집)

- 여행 장소 : 
첫째날 : 돌산공원
둘째날 : 여수 한화 아쿠아 플래닛, 여수 해상 케이블카, 오동도
마지막날 : 카페(모이핀)

#여수여행 #여수VLOG #여수맛집

—————————————————————————
*해당 컨텐츠의 조회수 광고수익중 매달 500,000원이 Save the Children 에 기부됩니다.

☁️ Contact Mail  
dailyhaneulinfo@gmail.com

☁️ Instagram
https://www.instagram.com/haneulina

☁️ 속옷쇼핑몰 하늘하늘
https://www.hn-hn.co.kr
[]
[]
['영상', '속', '정보', '마스크', '참존', '톤업핏', '베이지', '옷', '악세사리', '레이지코너', '숙소', '라테라스', '리조트', '쿠션', '라네즈', '네오', '쿠션', '맛집', '여행', '정보', '맛집', '첫째날', '황소식당', '돌게장', '맛집', '41번', '포차', '숙성회', '맛집', '둘째날', '조롱박', '아구찜', '서대회', '맛집', '호남갈비', '갈비맛집', '낭만', '포차', '거리', '셋째날', '꽃돌게장', '1번', '게장', '맛집', '여행', '장소', '첫째날', '돌산공원', '둘째날', '한화', '아쿠아', '플래닛', '해상', '케이블카', '오동', '마지막날', '카페', '모이핀', '여행', '맛집', '—————', '—————', '—————', '—————', '—————', '해당', '컨텐츠', '조회수', '광고

In [103]:
place_id = []
place_do =  []
place_si =[]
place_name = []
place_address = []
place_phone = []
place_url = []
place_X = []
place_Y = []
kakao_df = pd.DataFrame(columns=['ID','이름','주소','도', '시',
                                 '전화번호','URL','X좌표','Y좌표'])

url = "https://dapi.kakao.com/v2/local/search/keyword.json?" # kakaomap api에 접근
apikey = "c533ba443ba909af8d277567163d03fe"
sitename = youtube.channelname
for i in range(len(place)):    
    query = str(place[i])
    r = requests.get( url, params = {'query':query}, 
                     headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    
    # 장소를 입력하여 json파일 형태의 데이터 생성
    rj = r.json()
    # 입력했을 때 결과가 없으면 생략
    if rj['documents'] == []:                        
        continue
    if rj['documents'][0]['address_name'] in place_address:
        continue
    if rj['meta']['same_name']['selected_region'] == '':
        if sitename in rj['documents'][0]['address_name']:   # 검색 위치랑 같은 곳만 넣고 아니면 continue
            place_do.append(rj['documents'][0]['address_name'].split()[0])
            place_si.append(rj['documents'][0]['address_name'].split()[1])
        else:
            continue
    elif len(rj['meta']['same_name']['selected_region'].split()) == 1:
        if sitename in rj['documents'][0]['address_name']:
            place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
        else:
            continue
    else:
        if sitename in rj['documents'][0]['address_name']:
            place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
            place_si.append(rj['meta']['same_name']['selected_region'].split()[1])
        else :
            continue
    place_id.append(rj['documents'][0]['id'])         # 이후 list에 결과 append
    place_name.append(rj['documents'][0]['place_name'])
    place_address.append(rj['documents'][0]['address_name'])
    place_phone.append(rj['documents'][0]['phone'])
    place_url.append(rj['documents'][0]['place_url'])
    place_X.append(rj['documents'][0]['x'])
    place_Y.append(rj['documents'][0]['y'])
    
kakao_df=pd.DataFrame([place_id,place_name,place_address,place_do,place_si,place_phone,place_url,place_X,place_Y]).T
kakao_df.columns=['placeID','pname','paddress',
                  'pdo', 'psi','pphone','pdaumURL','pX','pY']
kakao_df


,placeID,pname,paddress,pdo,psi,pphone,pdaumURL,pX,pY
0,8169241,황소식당,전남 여수시 봉산동 269-1,전남,여수시,061-642-8007,http://place.map.kakao.com/8169241,127.72013656016564,34.73411992051962
1,27232447,41번포차,전남 여수시 봉산동 266-15,전남,여수시,061-642-8820,http://place.map.kakao.com/27232447,127.72148772372715,34.7336756480744
2,9445897,조롱박,전남 여수시 교동 366,전남,여수시,061-662-1733,http://place.map.kakao.com/9445897,127.73334157593006,34.740771859531556
3,24634263,원조여수서대회,전남 여수시 돌산읍 우두리 1041-7,전남,여수시,,http://place.map.kakao.com/24634263,127.746126684442,34.7210086105295
4,10984949,호남갈비,전남 여수시 공화동 1070-1,전남,여수시,061-662-3501,http://place.map.kakao.com/10984949,127.741875086966,34.7451451261449
5,1727173519,낭만카페,전남 여수시 고소동 291-1,전남,여수시,061-661-1188,http://place.map.kakao.com/1727173519,127.740358933392,34.7387328132004
6,26577684,꽃돌게장1번가,전남 여수시 봉산동 210-2,전남,여수시,061-644-0003,http://place.map.kakao.com/26577684,127.72611265489614,34.733022595376845
7,26515232,명동게장,전남 여수시 봉산동 273-35,전남,여수시,061-642-3735,http://place.map.kakao.com/26515232,127.72193767649,34.7329338358531
8,7843359,돌산공원,전남 여수시 돌산읍 우두리 799-76,전남,여수시,,http://place.map.kakao.com/7843359,127.739927808873,34.7305471491985
9,186032184,모이핀,전남 여수시 돌산읍 평사리 1273-5,전남,여수시,061-644-9313,http://place.map.kakao.com/186032184,127.77908440127192,34.687850799489055


In [57]:
for i in range(len(kakao_df)):   # places 테이블
    idx = f'{i+1}'
    sql = "insert into places values('" + idx + "','" + youtube.video_ids[n] + "','" + \
    kakao_df['placeID'][i] + "','" + kakao_df['pname'][i] + "','" + \
    kakao_df['paddress'][i] +"','" + kakao_df['pdo'][i] + "','" + \
    kakao_df['psi'][i] +"','" + kakao_df['pphone'][i] + "','" + kakao_df['pdaumURL'][i] +\
    "','" + kakao_df['pX'][i]+"','" + kakao_df['pY'][i] + "')"
    try:
        cur.execute(sql)
        con.commit()
    except:
        con.rollback()
        
con.close()